In [ ]:
# use this for loading pretrained model
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! apt-get install libavformat-dev libavdevice-dev
! pip install av==6.2.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libavformat-dev is already the newest version (7:3.4.8-0ubuntu0.2).
libavformat-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libavfilter-dev libpostproc-dev
The following NEW packages will be installed:
  libavdevice-dev libavfilter-dev libpostproc-dev
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,154 kB of archives.
After this operation, 5,444 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libpostproc-dev amd64 7:3.4.8-0ubuntu0.2 [51.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 libavfilter-dev amd64 7:3.4.8-0ubuntu0.2 [1,016 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd

In [ ]:
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar

--2021-04-30 04:29:15--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2021-04-30 04:29:15--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2124008126 (2.0G)
Saving to: ‘hmdb51_org.rar’

hmdb51_org.rar      100%[===================>]   1.98G  42.2MB/s    in 55s     

2021-04-30 04:30:11 (36.5 MB/s) - ‘hmdb51_org.rar’ saved [2124008126/2124008126]

--2021-04-30 04:30:11--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
R

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
from torch.autograd import Variable
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
from torchvision import get_video_backend
from torchvision.models.video import r3d_18 
from torchvision import transforms
import os
from tqdm.auto import tqdm
import numpy as np
import time
import av
import random
import shutil

from collections import OrderedDict

SEED = 491
torch.manual_seed(SEED)


In [ ]:
! mkdir -p video_data test_train_splits
! unrar e test_train_splits.rar test_train_splits
! rm test_train_splits.rar
! unrar e hmdb51_org.rar 
! rm hmdb51_org.rar
! mv *.rar video_data
for files in os.listdir('video_data'):
    foldername = files.split('.')[0]
    os.system("mkdir -p video_data/" + foldername)
    os.system("unrar e video_data/"+ files + " video_data/"+foldername)

! rm video_data/*.rar 


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from test_train_splits.rar

Extracting  test_train_splits/brush_hair_test_split1.txt                   0%  OK 
Extracting  test_train_splits/brush_hair_test_split2.txt                   1%  OK 
Extracting  test_train_splits/brush_hair_test_split3.txt                   1%  OK 
Extracting  test_train_splits/cartwheel_test_split1.txt                    2%  OK 
Extracting  test_train_splits/cartwheel_test_split2.txt                    2%  OK 
Extracting  test_train_splits/cartwheel_test_split3.txt                    3%  OK 
Extracting  test_train_splits/catch_test_split1.txt                        4%  OK 
Extracting  test_train_splits/catch_test_split2.txt                        4%  OK 
Extracting  test_train_splits/catch_test_split3.txt                        5%  OK 
Extracting  test_train_splits/chew_test_split1.txt              

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
print(len(GPUs))
gpu = GPUs[0]

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=5a2361cfe3775876df4869d66b98dd8d344ec545296d02c676e147e9b1777b3b
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
1


In [ ]:
! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
import transforms as T

--2021-04-28 05:19:16--  https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3102 (3.0K) [text/plain]
Saving to: ‘transforms.py’

transforms.py       100%[===================>]   3.03K  --.-KB/s    in 0s      

2021-04-28 05:19:16 (53.3 MB/s) - ‘transforms.py’ saved [3102/3102]



In [ ]:
val_split = 0.05
num_frames = 16 # 16
clip_steps = 50
num_workers = 2
pin_memory = True
train_tfms = torchvision.transforms.Compose([
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((128, 128)),
                                 T.RandomHorizontalFlip(),
                                 T.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),
                                 T.RandomCrop((112, 112))
                               ])  
test_tfms =  torchvision.transforms.Compose([
                                             T.ToFloatTensorInZeroOne(),
                                             T.Resize((128, 128)),
                                              T.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),
                                             T.CenterCrop((112, 112))
                                             ])
hmdb51_train = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=True,
                                                transform=train_tfms, num_workers=num_workers)


hmdb51_test = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=False,
                                                transform=test_tfms, num_workers=num_workers)
      
total_train_samples = len(hmdb51_train)
total_val_samples = round(val_split * total_train_samples)

print(f"number of train samples {total_train_samples}")
print(f"number of validation samples {total_val_samples}")
print(f"number of test samples {len(hmdb51_test)}")


number of train samples 7754
number of validation samples 388
number of test samples 3234


In [ ]:
bs = 4
lr = 1e-2
gamma = 0.1
step_size = 5
total_epochs = 10
config = {}

kwargs = {'num_workers':num_workers, 'pin_memory':True} if torch.cuda.is_available() else {'num_workers':num_workers}
#kwargs = {'num_workers':num_workers}
#kwargs = {}

hmdb51_train_v1, hmdb51_val_v1 = random_split(hmdb51_train, [total_train_samples - total_val_samples,
                                                                       total_val_samples])

#hmdb51_train_v1.video_clips.compute_clips(16, 1, frame_rate=30)
#hmdb51_val_v1.video_clips.compute_clips(16, 1, frame_rate=30)
#hmdb51_test.video_clips.compute_clips(16, 1, frame_rate=30)

#train_sampler = RandomClipSampler(hmdb51_train_v1.video_clips, 5)
#test_sampler = UniformClipSampler(hmdb51_test.video_clips, 5)
  
train_loader = DataLoader(hmdb51_train_v1, batch_size=32, shuffle=True, **kwargs)
val_loader   = DataLoader(hmdb51_val_v1, batch_size=16, shuffle=True, **kwargs)
test_loader  = DataLoader(hmdb51_test, batch_size=16, shuffle=False, **kwargs)


In [ ]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv3d(inp, oup, kernel_size=3, stride=stride, padding=(1, 1, 1), bias=False),
        nn.BatchNorm3d(oup),
        nn.ReLU6(inplace=True)
    )


def conv_1x1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv3d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm3d(oup),
        nn.ReLU6(inplace=True)
    )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride

        hidden_dim = round(inp * expand_ratio)
        self.use_res_connect = self.stride == (1, 1, 1) and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv3d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm3d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv3d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm3d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv3d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm3d(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv3d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm3d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv3d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm3d(oup),
            )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000, sample_size=224, width_mult=1.):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280
        inverted_residual_setting = [
            # t, c, n, s
            [1,  16, 1, (1,1,1)],
            [6,  24, 2, (2,2,2)],
            [6,  32, 3, (2,2,2)],
            [6,  64, 4, (2,2,2)],
            [6,  96, 3, (1,1,1)],
            [6, 160, 3, (2,2,2)],
            [6, 320, 1, (1,1,1)],
        ]

        # building first layer
        assert sample_size % 16 == 0.
        input_channel = int(input_channel * width_mult)
        self.last_channel = int(last_channel * width_mult) if width_mult > 1.0 else last_channel
        self.features = [conv_bn(3, input_channel, (1,2,2))]
        # building inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = int(c * width_mult)
            for i in range(n):
                stride = s if i == 0 else (1,1,1)
                self.features.append(block(input_channel, output_channel, stride, expand_ratio=t))
                input_channel = output_channel
        # building last several layers
        self.features.append(conv_1x1x1_bn(input_channel, self.last_channel))
        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.6),
            nn.Linear(self.last_channel, num_classes),
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = F.avg_pool3d(x, x.data.size()[-3:])
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.kernel_size[2] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


def get_fine_tuning_parameters(model, ft_portion):
    if ft_portion == "complete":
        return model.parameters()

    elif ft_portion == "last_layer":
        ft_module_names = []
        ft_module_names.append('classifier')

        parameters = []
        for k, v in model.named_parameters():
            for ft_module in ft_module_names:
                if ft_module in k:
                    parameters.append({'params': v})
                    break
            else:
                parameters.append({'params': v, 'lr': 0.0})
        return parameters

    else:
        raise ValueError("Unsupported ft_portion: 'complete' or 'last_layer' expected")

    
def get_model(**kwargs):
    """
    Returns the model.
    """
    model = MobileNetV2(**kwargs)
    return model



In [ ]:
# model = get_model(num_classes=51, sample_size=112, width_mult=1.)
# if (torch.cuda.is_available()):
#   model = model.cuda()
# model = nn.DataParallel(model, device_ids=None)
# # print(model)
# input_var = Variable(torch.randn(8, 3, 16, 112, 112))
# output = model(input_var)
# print(output.shape)

In [ ]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def update_acc(self, val, n=1):
        self.val = val/n
        self.sum += val
        self.count += n
        self.avg = self.sum / self.count

def calculate_accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def save_checkpoint(state, is_best,path):
    torch.save(state, '%s' % (path))
    if is_best:
        shutil.copyfile('%s' % (path),'HAR_best.pth')



In [ ]:
def train_epoch(epoch, data_loader, model, optimizer):
    print('train at epoch {}'.format(epoch))

    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    end_time = time.time()
    for i, data in enumerate(data_loader):
        inputs, targets = data[0], data[-1]
        data_time.update(time.time() - end_time)

        if torch.cuda.is_available():
          targets = targets.cuda()
          inputs = inputs.cuda()
        inputs = Variable(inputs)
        targets = Variable(targets)
        outputs = model(inputs)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, targets)

        losses.update(loss.data, inputs.size(0))
        prec1, prec5 = calculate_accuracy(outputs.data, targets.data, topk=(1,5))
        top1.update(prec1, inputs.size(0))
        top5.update(prec5, inputs.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end_time)
        end_time = time.time()

        if i % 10 ==0:
            print('Epoch: [{0}][{1}/{2}]\t lr: {lr:.5f}\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.5f} ({top1.avg:.5f})\t'
                  'Prec@5 {top5.val:.5f} ({top5.avg:.5f})'.format(
                  epoch,
                  i,
                  len(data_loader),
                  batch_time=batch_time,
                  data_time=data_time,
                  loss=losses,
                  top1=top1,
                  top5=top5,
                  lr=optimizer.param_groups[0]['lr']))

    

In [ ]:
def val_epoch(epoch, data_loader, model):
  print('validation at epoch {}'.format(epoch))

  model.eval()

  batch_time = AverageMeter()
  data_time = AverageMeter()
  losses = AverageMeter()
  top1 = AverageMeter()
  top5 = AverageMeter()

  end_time = time.time()
  for i, data in enumerate(data_loader):
      inputs, targets = data[0], data[-1]
      data_time.update(time.time() - end_time)

      if torch.cuda.is_available():
          targets = targets.cuda()
          inputs = inputs.cuda()
      with torch.no_grad():
          inputs = Variable(inputs)
          targets = Variable(targets)
      outputs = model(inputs)
      criterion = nn.CrossEntropyLoss()
      loss = criterion(outputs, targets)
      prec1, prec5 = calculate_accuracy(outputs.data, targets.data, topk=(1,5))
      top1.update(prec1, inputs.size(0))
      top5.update(prec5, inputs.size(0))

      losses.update(loss.data, inputs.size(0))

      batch_time.update(time.time() - end_time)
      end_time = time.time()

      print('Epoch: [{0}][{1}/{2}]\t'
            'Time {batch_time.val:.5f} ({batch_time.avg:.5f})\t'
            'Data {data_time.val:.5f} ({data_time.avg:.5f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
            'Prec@1 {top1.val:.5f} ({top1.avg:.5f})\t'
            'Prec@5 {top5.val:.5f} ({top5.avg:.5f})'.format(
            epoch,
            i + 1,
            len(data_loader),
            batch_time=batch_time,
            data_time=data_time,
            loss=losses,
            top1=top1,
            top5=top5))
  return losses.avg.item(), top1.avg.item()

In [ ]:
def test(data_loader, model):
    print('test')

    model.eval()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end_time = time.time()
    
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            inputs, targets = data[0], data[-1]
            data_time.update(time.time() - end_time)
            if torch.cuda.is_available():
                targets = targets.cuda()
                inputs = inputs.cuda()
           
            inputs = Variable(inputs)
            targets = Variable(targets)
            outputs = model(inputs)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(outputs, targets)
            prec1, prec5 = calculate_accuracy(outputs.data, targets.data, topk=(1,5))
            top1.update(prec1, inputs.size(0))
            top5.update(prec5, inputs.size(0))

            losses.update(loss.data, inputs.size(0))

            batch_time.update(time.time() - end_time)
            end_time = time.time()

            print('[{0}/{1}]\t'
            'Time {batch_time.val:.5f} ({batch_time.avg:.5f})\t'
            'Data {data_time.val:.5f} ({data_time.avg:.5f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
            'Prec@1 {top1.val:.5f} ({top1.avg:.5f})\t'
            'Prec@5 {top5.val:.5f} ({top5.avg:.5f})'.format(
            i + 1,
            len(data_loader),
            batch_time=batch_time,
            data_time=data_time,
            loss=losses,
            top1=top1,
            top5=top5))
            

In [ ]:
path = '/content/drive/MyDrive/HAR.pth'

model = get_model(num_classes=51, sample_size=112, width_mult=1.)
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['state_dict'])
if torch.cuda.is_available():
  model = model.cuda()

optimizer = torch.optim.SGD(
            model.parameters(),
            lr=lr,
            momentum=0.9,
            dampening=0.9,
            weight_decay=1e-3)
scheduler = ReduceLROnPlateau(
            optimizer, 'min', patience=7)

optimizer.load_state_dict(checkpoint['optimizer'])
best_prec1 = checkpoint['best_prec1']

In [ ]:
for param in model.parameters():
    param.requires_grad = False
for i in range(18, 19):
    for param in model.features[i].parameters():
      param.requires_grad = True
for param in model.classifier.parameters():
     param.requires_grad = True
print(sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6)

0.477491


0.001


In [ ]:

print("Launching Action Recognition Model training")
for i in range(30):
  train_epoch(i, train_loader, model, optimizer)
  state = {
      'state_dict': model.state_dict(),
      'optimizer': optimizer.state_dict(),
      'best_prec1': best_prec1
      }
  save_checkpoint(state, False, path)
  validation_loss, prec1 = val_epoch(i, val_loader, model)
  is_best = prec1 > best_prec1
  best_prec1 = max(prec1, best_prec1)
  state = {
  'state_dict': model.state_dict(),
  'optimizer': optimizer.state_dict(),
  'best_prec1': best_prec1
  }
  save_checkpoint(state, is_best, path)

Launching Action Recognition Model training
train at epoch 0


/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


Epoch: [0][0/231]	 lr: 0.00100	Time 3.951 (3.951)	Data 3.505 (3.505)	Loss 1.0564 (1.0564)	Prec@1 68.75000 (68.75000)	Prec@5 93.75000 (93.75000)
Epoch: [0][10/231]	 lr: 0.00100	Time 2.436 (1.860)	Data 2.089 (1.534)	Loss 0.9747 (1.0657)	Prec@1 71.87500 (71.59091)	Prec@5 96.87500 (92.32955)
Epoch: [0][20/231]	 lr: 0.00100	Time 1.794 (1.758)	Data 1.530 (1.463)	Loss 1.1200 (1.0888)	Prec@1 68.75000 (70.23810)	Prec@5 93.75000 (91.66667)
Epoch: [0][30/231]	 lr: 0.00100	Time 2.219 (1.747)	Data 1.920 (1.462)	Loss 1.1291 (1.1202)	Prec@1 71.87500 (68.75000)	Prec@5 87.50000 (91.83467)
Epoch: [0][40/231]	 lr: 0.00100	Time 1.958 (1.728)	Data 1.692 (1.443)	Loss 1.2424 (1.1500)	Prec@1 62.50000 (68.21646)	Prec@5 90.62500 (91.31097)
Epoch: [0][50/231]	 lr: 0.00100	Time 1.443 (1.712)	Data 1.188 (1.431)	Loss 1.2577 (1.1638)	Prec@1 68.75000 (68.07598)	Prec@5 90.62500 (90.62500)
Epoch: [0][60/231]	 lr: 0.00100	Time 1.989 (1.713)	Data 1.763 (1.434)	Loss 1.2781 (1.1462)	Prec@1 62.50000 (68.74999)	Prec@5 84.375

/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


Epoch: [0][1/25]	Time 2.21053 (2.21053)	Data 1.96228 (1.96228)	Loss 1.2960 (1.2960)	Prec@1 68.75000 (68.75000)	Prec@5 87.50000 (87.50000)
Epoch: [0][2/25]	Time 0.17888 (1.19471)	Data 0.00145 (0.98187)	Loss 1.7348 (1.5154)	Prec@1 62.50000 (65.62500)	Prec@5 87.50000 (87.50000)
Epoch: [0][3/25]	Time 1.78994 (1.39312)	Data 1.58739 (1.18371)	Loss 0.9016 (1.3108)	Prec@1 75.00000 (68.75000)	Prec@5 87.50000 (87.50000)
Epoch: [0][4/25]	Time 0.19323 (1.09315)	Data 0.00114 (0.88807)	Loss 0.5851 (1.1294)	Prec@1 81.25000 (71.87500)	Prec@5 100.00000 (90.62500)
Epoch: [0][5/25]	Time 1.53290 (1.18110)	Data 1.35941 (0.98234)	Loss 1.6266 (1.2288)	Prec@1 62.50000 (70.00000)	Prec@5 75.00000 (87.50000)
Epoch: [0][6/25]	Time 0.13677 (1.00704)	Data 0.00143 (0.81885)	Loss 2.1710 (1.3858)	Prec@1 50.00000 (66.66667)	Prec@5 75.00000 (85.41667)
Epoch: [0][7/25]	Time 1.56891 (1.08731)	Data 1.46806 (0.91160)	Loss 1.4517 (1.3952)	Prec@1 37.50000 (62.50000)	Prec@5 87.50000 (85.71429)
Epoch: [0][8/25]	Time 0.17089 (0.

...

Epoch: [12][0/231]	 lr: 0.00100	Time 3.859 (3.859)	Data 3.509 (3.509)	Loss 0.9095 (0.9095)	Prec@1 75.00000 (75.00000)	Prec@5 93.75000 (93.75000)
Epoch: [12][10/231]	 lr: 0.00100	Time 2.938 (1.856)	Data 2.587 (1.567)	Loss 1.1382 (1.2371)	Prec@1 75.00000 (66.47727)	Prec@5 87.50000 (88.06818)
Epoch: [12][20/231]	 lr: 0.00100	Time 2.233 (1.758)	Data 1.880 (1.459)	Loss 0.8898 (1.1598)	Prec@1 78.12500 (68.89881)	Prec@5 93.75000 (88.98810)
Epoch: [12][30/231]	 lr: 0.00100	Time 2.625 (1.752)	Data 2.274 (1.456)	Loss 1.4315 (1.1909)	Prec@1 56.25000 (67.33871)	Prec@5 90.62500 (89.31451)
Epoch: [12][40/231]	 lr: 0.00100	Time 1.834 (1.719)	Data 1.598 (1.430)	Loss 1.2309 (1.1541)	Prec@1 65.62500 (68.52134)	Prec@5 87.50000 (89.93902)
Epoch: [12][50/231]	 lr: 0.00100	Time 0.998 (1.698)	Data 0.766 (1.413)	Loss 1.0717 (1.1553)	Prec@1 62.50000 (67.95343)	Prec@5 96.87500 (90.01226)
Epoch: [12][60/231]	 lr: 0.00100	Time 0.828 (1.695)	Data 0.619 (1.408)	Loss 1.3737 (1.1551)	Prec@1 68.75000 (67.98155)	Prec@5

/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


Epoch: [12][1/25]	Time 2.18399 (2.18399)	Data 1.99388 (1.99388)	Loss 1.0105 (1.0105)	Prec@1 68.75000 (68.75000)	Prec@5 93.75000 (93.75000)
Epoch: [12][2/25]	Time 0.19580 (1.18990)	Data 0.00163 (0.99775)	Loss 1.1943 (1.1024)	Prec@1 62.50000 (65.62500)	Prec@5 87.50000 (90.62500)
Epoch: [12][3/25]	Time 1.73769 (1.37249)	Data 1.54061 (1.17871)	Loss 0.6737 (0.9595)	Prec@1 87.50000 (72.91667)	Prec@5 93.75000 (91.66667)
Epoch: [12][4/25]	Time 0.11713 (1.05865)	Data 0.00950 (0.88640)	Loss 0.7851 (0.9159)	Prec@1 75.00000 (73.43750)	Prec@5 100.00000 (93.75000)
Epoch: [12][5/25]	Time 1.78956 (1.20483)	Data 1.58373 (1.02587)	Loss 1.3813 (1.0090)	Prec@1 68.75000 (72.50000)	Prec@5 81.25000 (91.25000)
Epoch: [12][6/25]	Time 0.16018 (1.03073)	Data 0.00130 (0.85511)	Loss 0.7966 (0.9736)	Prec@1 87.50000 (75.00000)	Prec@5 93.75000 (91.66667)
Epoch: [12][7/25]	Time 1.57091 (1.10790)	Data 1.39194 (0.93180)	Loss 0.7335 (0.9393)	Prec@1 81.25000 (75.89286)	Prec@5 93.75000 (91.96429)
Epoch: [12][8/25]	Time 0.1

/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


Epoch: [13][0/231]	 lr: 0.00100	Time 3.470 (3.470)	Data 3.121 (3.121)	Loss 1.2517 (1.2517)	Prec@1 59.37500 (59.37500)	Prec@5 87.50000 (87.50000)
Epoch: [13][10/231]	 lr: 0.00100	Time 2.660 (1.845)	Data 2.261 (1.551)	Loss 1.3593 (1.2386)	Prec@1 59.37500 (66.19318)	Prec@5 84.37500 (88.35227)
Epoch: [13][20/231]	 lr: 0.00100	Time 2.710 (1.772)	Data 2.322 (1.470)	Loss 1.4534 (1.2031)	Prec@1 65.62500 (66.66666)	Prec@5 87.50000 (89.58334)
Epoch: [13][30/231]	 lr: 0.00100	Time 2.187 (1.721)	Data 1.833 (1.425)	Loss 1.3210 (1.1985)	Prec@1 62.50000 (67.43951)	Prec@5 87.50000 (89.31451)
Epoch: [13][40/231]	 lr: 0.00100	Time 1.733 (1.702)	Data 1.480 (1.414)	Loss 0.9369 (1.1755)	Prec@1 78.12500 (67.75914)	Prec@5 93.75000 (89.40549)
Epoch: [13][50/231]	 lr: 0.00100	Time 3.029 (1.712)	Data 2.707 (1.420)	Loss 1.0578 (1.1827)	Prec@1 71.87500 (67.03432)	Prec@5 87.50000 (89.52206)
Epoch: [13][60/231]	 lr: 0.00100	Time 2.503 (1.696)	Data 2.186 (1.402)	Loss 1.1544 (1.1785)	Prec@1 81.25000 (67.62295)	Prec@5

In [ ]:
test(test_loader, model)

test


/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:159: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


[1/203]	Time 2.09938 (2.09938)	Data 1.84501 (1.84501)	Loss 0.1146 (0.1146)	Prec@1 100.00000 (100.00000)	Prec@5 100.00000 (100.00000)
[2/203]	Time 0.20091 (1.15014)	Data 0.01170 (0.92835)	Loss 0.7601 (0.4373)	Prec@1 93.75000 (96.87500)	Prec@5 100.00000 (100.00000)
[3/203]	Time 1.35848 (1.21959)	Data 1.16585 (1.00752)	Loss 1.4729 (0.7825)	Prec@1 62.50000 (85.41667)	Prec@5 68.75000 (89.58334)
[4/203]	Time 0.14888 (0.95191)	Data 0.00148 (0.75601)	Loss 4.2379 (1.6464)	Prec@1 18.75000 (68.75000)	Prec@5 25.00000 (73.43750)
[5/203]	Time 1.49552 (1.06063)	Data 1.31444 (0.86769)	Loss 3.3013 (1.9774)	Prec@1 37.50000 (62.50000)	Prec@5 37.50000 (66.25000)
[6/203]	Time 0.19410 (0.91621)	Data 0.00146 (0.72332)	Loss 2.1286 (2.0026)	Prec@1 50.00000 (60.41667)	Prec@5 68.75000 (66.66667)
[7/203]	Time 1.69205 (1.02704)	Data 1.54209 (0.84029)	Loss 6.4670 (2.6403)	Prec@1 12.50000 (53.57143)	Prec@5 12.50000 (58.92857)
[8/203]	Time 0.19977 (0.92364)	Data 0.00840 (0.73630)	Loss 5.4991 (2.9977)	Prec@1 0.00000 (